# Desafio Properati - Limpieza de datos - Grupo 3

En este proyecto el desafío es limpiar la base de datos de inmuebles provista por Properati.

El objetivo de la limpieza es dejar listo el dataset para luego poder utilizarlo para hacer regresiones y calcular el valor de nuevas observaciones.

## ¿Cómo lo vamos a hacer?
Decidimos estructurar nuestras tareas en cuatro bloques de trabajo:
* 1: Análisis exploratorio. 
* 2: Normalizar, corregir y rellenar la informacion que lo permita, sin afectar prediciones futuras.
* 3: Quitar todo lo que no nos sirve.
* 4: Calcular las variables dummies y mostrar los resultados.

## 1. Análisis exploratorio

A partir del analisis exploratorio de los datos, ponemos a prueba algunas de las hipótesis que tendremos en cuenta para estandarizar la información. 
En los casos en los que nuestras hipótesis se corroboran, definimos las estrategias que tomaremos para corregir el dataset en el siguiente bloque de trabajo. 

In [1]:
# Importo librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re

# usado para pruebas hechas sobre  las urls de imagenes y link a las publicaciones
import requests 
import hashlib 



from IPython.core.display import HTML
%matplotlib inline

In [2]:
# importo archivo
df = pd.read_csv("properatti.csv")
df.shape

(121220, 26)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

In [4]:
df.describe()

,Unnamed: 0,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,121220.000000,1.025030e+05,69670.000000,69670.000000,1.008100e+05,1.008100e+05,1.008100e+05,81892.000000,101313.000000,68617.000000,8.765800e+04,7899.000000,47390.000000,1.426200e+04
mean,60609.500000,3.574442e+06,-34.626210,-59.266290,4.685259e+05,4.229397e+06,2.397006e+05,233.795328,133.050181,2160.086916,6.912216e+03,17.452336,3.080840,5.009234e+03
std,34993.344153,3.541306e+05,1.980936,2.299922,2.260101e+06,6.904714e+06,3.913239e+05,1782.222147,724.351479,2759.288621,2.837864e+04,120.243621,1.860773,1.204403e+05
min,0.000000,3.427208e+06,-54.823985,-75.678931,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.600000,1.510204e+00,1.000000,1.000000,1.000000e+00
25%,30304.750000,3.430234e+06,-34.669065,-58.727040,1.100000e+05,1.583309e+06,8.973388e+04,50.000000,45.000000,1218.181818,1.550000e+03,1.000000,2.000000,1.000000e+03
50%,60609.500000,3.433910e+06,-34.597985,-58.480128,1.850000e+05,2.558452e+06,1.450000e+05,84.000000,75.000000,1800.000000,2.213115e+03,3.000000,3.000000,2.000000e+03
75%,90914.250000,3.836668e+06,-34.441299,-58.395908,4.200000e+05,4.675792e+06,2.650000e+05,200.000000,150.000000,2486.411765,3.355549e+03,6.000000,4.000000,4.000000e+03
max,121219.000000,6.948895e+06,4.545843,-53.733330,6.500000e+08,8.212711e+08,4.654544e+07,200000.000000,187000.000000,206333.333333,4.000000e+06,3150.000000,32.000000,1.000150e+07


In [5]:
#Obtenemos los porcentajes de datos faltantes de cada columna
for cols in df.columns:
    nulos = df[cols].isnull().sum()
    porcentaje = nulos/len(df)
    print(f'{porcentaje*100:.0f}%', cols)

0% Unnamed: 0
0% operation
0% property_type
0% place_name
0% place_with_parent_names
0% country_name
0% state_name
15% geonames_id
43% lat-lon
43% lat
43% lon
17% price
17% currency
17% price_aprox_local_currency
17% price_aprox_usd
32% surface_total_in_m2
16% surface_covered_in_m2
43% price_usd_per_m2
28% price_per_m2
93% floor
61% rooms
88% expenses
0% properati_url
0% description
0% title
3% image_thumbnail


### Trabajos exploratorios con la superficie de las propiedades:

##### Trabajo  sobre m2 en superficie total y superficie cubierta con valores invertidos:

In [6]:
# Buscamos los casos en los que la superficie total es menor que la superficie cubierta.
print("Muestra de M2 con superficie total menor que superficie cubierta")
display(df.loc[(df.surface_total_in_m2 < df.surface_covered_in_m2),["surface_total_in_m2","surface_covered_in_m2"]].sample(5))


#Suponemos que los valores de las columnas de superficie total y superficie cubierta pueden estar invertidos por error. 
#Estimamos la división de uno sobre otro para calcular la media de esta diferencia que nos permita corroborar nuestra hipótesis.
df['cubierta_sobre_total'] = df['surface_total_in_m2']/ df['surface_covered_in_m2'] 
df['total_sobre_cubierta'] = df['surface_covered_in_m2']/ df['surface_total_in_m2']
df['valores_invertidos'] = df['surface_covered_in_m2'] < df['surface_total_in_m2']

#Dropeamos los valores iguales para que no afecten el promedio.
#Observamos que son valores similares y que por lo tanto nos permiten asumir que los valores de ambas columnas fueron invertidos.
print("Resumen de la media de la relacion de variables sobre M2 (valores invertidos y no invertidos)")
display(
    df.drop(df.loc[df['surface_total_in_m2'] == df['surface_covered_in_m2']].index)\
    [["valores_invertidos","total_sobre_cubierta","cubierta_sobre_total"]].groupby(['valores_invertidos']).mean()
)

# Por esto decidimo invertir los valores de las columnas de superficie total y superficie cubierta en aquellos casos que la primera es inferior a la segunda

Muestra de M2 con superficie total menor que superficie cubierta


,surface_total_in_m2,surface_covered_in_m2
6918,100.0,130.0
65225,1000.0,1100.0
23829,60.0,120.0
33266,463.0,580.0
55249,381.0,436.0


Resumen de la media de la relacion de variables sobre M2 (valores invertidos y no invertidos)


,total_sobre_cubierta,cubierta_sobre_total
valores_invertidos,,
False,5.471169,0.696969
True,0.737385,inf


##### Trabajo sobre m2 en superficie total y superficie cubierta según tipo de propiedad:

In [17]:
#Contamos los casos que tienen una superficie total inferior a superficie cubierta según tipo de propiedad.
print ('Casos con superifice total inferior a superficie cubierta por tipo de propiedad')
display (df.loc[(df.surface_total_in_m2 < df.surface_covered_in_m2)]['property_type'].value_counts())

print ('--------------')

#Contamos casos segun tipo de propiedad para obtener relacion porcentual. Concluimos que no es una variable significativa para etsa relación.
print ('Casos totales por tipo de propiedad')
display (df['property_type'].value_counts())


Casos con superifice total inferior a superficie cubierta por tipo de propiedad


house        666
apartment    351
store         47
PH            42
Name: property_type, dtype: int64

--------------
Casos totales por tipo de propiedad


apartment    71065
house        40268
PH            5751
store         4136
Name: property_type, dtype: int64

##### Trabajo  para completar m2 a partir del valor de la propiedad y del valor por metro:

In [ ]:
# Solo puedo averiguar mi incognita si tengo metros y valor por metro
# ejemplo: x = df['price_aprox_usd']/df['price_usd_per_m2']

#Buscamos las diferentes combinaciones
print("USD, Con Precio y PPM USD pero sin M2: {}".format(
    df.loc[(~df["price_aprox_usd"].isnull()) & (~df["price_usd_per_m2"].isnull()) & (df["surface_total_in_m2"].isnull()),"operation"].count()
))
print("ARS, Con Precio y PPM ARS pero sin M2: {}".format(
    df.loc[(~df["price_aprox_local_currency"].isnull()) & (~df["price_per_m2"].isnull()) & (df["surface_total_in_m2"].isnull()),"operation"].count()
))
print("Con Precio default y PPM default pero sin M2: {}".format(
    df.loc[(~df["price"].isnull()) & (~df["price_per_m2"].isnull()) & (df["surface_total_in_m2"].isnull()),"operation"].count()
))
print("Con Precio default y PPM USD pero sin M2: {}".format(
    df.loc[(~df["price"].isnull()) & (~df["price_usd_per_m2"].isnull()) & (df["surface_total_in_m2"].isnull()),"operation"].count()
))

# Hipotesis refutada, no sirve para obtener nuevos datos

##### Trabajo para completar m2 a partir de valores útiles en título y descripción:

In [32]:
#Creamos una regex  y la corremos en titulo y en descripcion para ver que encuentra
pattern= r'([\.\d]{2,99}) (?!m²|m2|mt|metro)'
m2ExtractedFromTitle=df.loc[df["surface_total_in_m2"].isnull(),'title'].str.extract(pattern, re.IGNORECASE)
m2FromDescription=df.loc[df["surface_total_in_m2"].isnull(),'description'].str.extract(pattern, re.IGNORECASE)

# Imprimir resumen resultados
print("Valores en columna titulo: {}".format(m2ExtractedFromTitle.dropna().describe().loc["count",0]))
print("Valores en columna descripcion: {}".format(m2FromDescription.dropna().describe().loc["count",0]))


# Imprimir lo encontrado en titulo.
df["m2Extracted"] = m2ExtractedFromTitle
for index,x in df.iloc[m2ExtractedFromTitle.dropna().index].loc[:,["title","m2Extracted"]].iterrows():
    print("\r Found: {}  \t Title: {}  ".format(x["m2Extracted"],x["title"]))

    
# Dropeamos columna temporal
df.drop("m2Extracted",axis=1,inplace=True)

#Tras revisar los resultados, hay muchas informacion falsa y no es confiable

Valores en columna titulo: 961
Valores en columna descripcion: 7440
 Found: 36  	 Title: VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37                 
 Found: 7637  	 Title: WHITE 7637 - 2 DORMITORIOS CON PATIO        
 Found: 237  	 Title: EXCELENTE | MENDOZA 237 | MTS PLAZA COLO  
 Found: 300  	 Title: Rosario - San Nicolas 300 - Venta Deptos 1 Dor estrenar Escalera - InmGazze  
 Found: 3400  	 Title: Rosario - Tucuman 3400 - Venta Dptos 1 Dormitorio en construcción - InmGazze  
 Found: 3400  	 Title: Rosario - Tucuman 3400 - Venta Dptos 2 Dormitorios en construcción - InmGazze  
 Found: 505  	 Title: 7 e/505 y 506  
 Found: 1431  	 Title: Calle 8 1431 (61 y 62)  1400  
 Found: 135  	 Title: 135 e/ 531 y 32  
 Found: 62  	 Title: 62 e/ 12 y 13 800  
 Found: 27  	 Title: ITALIA Y 27 FEBRERO INSUPERABLE CASA 2 DORMIT. PATIO PARRILLERO TERRAZA 86 MTR2  
 Found: 12  	 Title: GARITA 12 SALIDA A 2 CALLES Y RUTA 9 MODERNA  CASA SOBRE 1600mt.2 TERRENO   
 Found: 14  	 Title: FUNES GARITA 14 C

 Found: 3225  	 Title: Brown Alte. 3225 8º B  
 Found: 489  	 Title: Casa en 489 e/ Gral Belgrano y 21  
 Found: 400  	 Title: Eugenio Garzon 400 - 1 DOR - A metros de Patio Olmos - ESCRITURA!!  
 Found: 20  	 Title: INVERSOR - ZONA D - ALTA CORDOBA - PARA 20 DEPARTAMENTOS  
 Found: 7.30  	 Title: Local c losa y entrepiso sobre av vergara, de 7.30 x 21 mts, con baño mas entrepiso de  7.3 x 6. venta directa  
 Found: 40  	 Title: Depto  APTO BANCO en 2 e/ 40 y 41  
 Found: 200  	 Title: CHACABUCO al 200 -CENTRO- con escritura muy amplio  
 Found: 449  	 Title: Casa en 21a e/ 449 y 450  
 Found: 1100  	 Title: CHACABUCO 1100 - CATEGORIA ABSOLUTA - CONSULTA DISPONIBILIDAD  
 Found: 28  	 Title: casa en 28 e/ 479 y 480  
 Found: 49  	 Title: CASA APTA BANCO EN 49 E/ 154 Y 155  
 Found: 12  	 Title: Casa en 12 y 51  
 Found: 1900  	 Title: DEPARTAMENTO DE 1 DORMITORIO - SANTA ROSA 1900 ALBERDI  
 Found: 493  	 Title: Casa en 493 e/ 22 y 23  
 Found: 3000  	 Title: Casa 4 DORM PATIO PILETA Y

##### Trabajo  para completar cantidad de ambientes con valores útiles en título y descripción:

In [29]:
#Creamos una regex  y la corremos en título y en descripcíon para ver que encuentra
pat_ambientes = r'\b(\d{1,2})\s*amb'

#Extraemos los datos las nuevas columnas amb_tit y amb_desc del título y la descripcion
df['amb_tit'] = df.title.str.extract(pat_ambientes, re.IGNORECASE, expand=True).astype(np.float)
df['amb_desc'] = df.description.str.extract(pat_ambientes, re.IGNORECASE, expand=True).astype(np.float)

#Verificamos los valores extraídos
print ("Cantidad de ambientes extraídos de títulos:")
display (df.loc[~(df.amb_tit.isnull())].filter(['amb_tit']).sort_values('amb_tit').amb_tit.unique())
print ("Cantidad de ambientes extraídos de descripciones:")
display (df.loc[~(df.amb_desc.isnull())].filter(['amb_desc']).sort_values('amb_desc').amb_desc.unique())

#Tras revisar los resultados,  concluimos que en las descripciones hay mucha información falsa y no confiable. 
#Optamos solo por extraer los valores del título. 

Cantidad de ambientes extraídos de títulos:


array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 14.,
       15.])

Cantidad de ambientes extraídos de descripciones:


array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       14., 16., 17., 18., 20., 22., 23., 24., 25., 28., 32., 36., 37.,
       50., 53., 65., 70., 75., 84., 90.])

##### Trabajo para completar cantidad de dormitorios con valores útiles en título y descripción:

In [30]:
#Creamos una regex  y la corremos en título y en descripción para ver que encuentra
pat_dormitorios = r'\b(\d{1,2})\s*dor'

#Extraemos los datos a las nuevas columnas dor_tit y dor_desc del título y la descripcion
df['dor_tit'] = df.title.str.extract(pat_dormitorios, re.IGNORECASE, expand=True).astype(np.float)
df['dor_desc'] = df.description.str.extract(pat_dormitorios, re.IGNORECASE, expand=True).astype(np.float)

#Verificamos los valores extraídos
print ("Cantidad de dormitorios extraídos de títulos:")
display (df.loc[~(df.dor_tit.isnull())].filter(['dor_tit']).sort_values('dor_tit').dor_tit.unique())
print ("Cantidad de dormitorios extraídos de descripciones:")
display (df.loc[~(df.dor_desc.isnull())].filter(['dor_desc']).sort_values('dor_desc').dor_desc.unique()
)
#Tras revisar los resultados, optamos por no tomar ninguno de los datos extraidos dado que mucha información es falsa y no confiable

Cantidad de dormitorios extraídos de títulos:


array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.])

Cantidad de dormitorios extraídos de descripciones:


array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 13.,
       14., 15., 18., 20., 22., 24., 25., 27., 30., 31., 32., 33., 34.,
       35., 40., 41., 42., 43., 44., 45., 48., 50., 51., 52., 53., 54.,
       57., 60., 62., 65., 70., 72., 74., 75., 76., 77., 78., 80., 83.,
       84., 85., 87., 90., 93., 95.])

##### Trabajo para recuperar cantidad de baños con valores útiles en título y descripción: 

In [31]:
#Creamos una regex  y la corremos en título y en descripción para ver que encuentra
pat_banos = r'\b(\d{1,2})\s*bañ'

#Extraemos los datos a las nuevas columnas bath_tit y bath_desc del título y la descripcion
df['bath_tit'] = df.title.str.extract(pat_banos, re.IGNORECASE, expand=True).astype(np.float)
df['bath_desc'] = df.description.str.extract(pat_banos, re.IGNORECASE, expand=True).astype(np.float)

#Verificamos los valores extraídos
print ("Cantidad de baños extraídos de títulos:")
display (df.loc[~(df.bath_tit.isnull())].filter(['bath_tit']).sort_values('bath_tit').bath_tit.unique())
print ("Cantidad de baños extraídos de descripciones:")
display (df.loc[~(df.bath_desc.isnull())].filter(['bath_desc']).sort_values('bath_desc').bath_desc.unique())

#Tras revisar los resultados, optamos por no tomar ninguno de los datos extraidos dado que el volumen de datos que extraemos no es significativo

Cantidad de baños extraídos de títulos:


array([1., 2., 3., 4., 5., 6.])

Cantidad de baños extraídos de descripciones:


array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       15., 16., 18., 20., 22., 23., 25., 27., 30., 31., 36., 37., 40.,
       41., 42., 43., 48., 50., 51., 52., 55., 60., 61., 64., 65., 70.,
       71., 75., 76., 80., 82., 85., 86., 90., 93., 95.])

##### Trabajo para completar los datos faltantes de 'rooms' a partir de los datos de ambientes obtenidos en títulos:

In [20]:
#Tomamos un subset compuesto por las observaciones que tienen valores en la variable `rooms` y los que también tienen en el título, es decir `amb_tit` y hacemos un booleano entre dichas Series.
#Lo que nos devuelve es una serie de booleanos, que tienen la propiedad de que los valores `True` son iguales a 1, mientras que los `False` son 0.
a = df.filter(['amb_tit', 'dor_tit', 'rooms']).\
        loc[~(df.rooms.isnull()) & ~(df.amb_tit.isnull())].amb_tit.astype(np.float64) == \
    df.filter(['amb_tit', 'dor_tit', 'rooms']).loc[~(df.rooms.isnull()) & ~(df.amb_tit.isnull())].rooms

#Hacemos la suma total de los valores, donde nos dará el total de `True`s, y lo dividimos en la longitud total de la serie.
a.sum()/len(a)

#El resultado anterior indica que rooms y ambientes es lo mismo en un 94% de los casos, por lo que podemos rellenar los `NaN` de *'rooms'* con *'amb_tit'*


0.9498057734785589

### Trabajos exploratiorios con precios de las propiedades:

In [ ]:
# Me centrare en las columnas 'place_name', 'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2'
# Tenemos 20 mil casos sin ningun dato de precio. Creo el DF dfprecio para trabajar todo lo relacionado a precio en el:
mycolums = df[['place_name', 'price', 'currency', 'price_aprox_local_currency',
               'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
               'price_usd_per_m2', 'price_per_m2', 'title', 'description']];
dfprecio = mycolums.loc[((df['price'].isnull()) & (df['price_aprox_usd'].isnull())\
                         &(df['price_aprox_local_currency'].isnull()) \
                         &(df['price_usd_per_m2'].isnull()) & (df['price_per_m2'].isnull()))];

# Extraigo precios de los titulos y descripciones. Creo 2 dataframes
patron_p = r'([$]|[Uu][Ss$][$SDsd]*)\s*(\d*)[\s* .,]*(\d*)[\s* .,](\d*)'
df_precio_desc = dfprecio['description'].str.extract(patron_p)
df_precio_title = dfprecio['title'].str.extract(patron_p)


### Trabajos exploratorios con la ubicación de las propiedades: 

###### Trabajo para recuperar ubicación faltante de las propiedades desde geonames

In [21]:
# Descargamos de acá: https://download.geonames.org/export/dump/ la base de datos de geonames de argentina
#Leemos la base de datos
geo_column_names = ["geonameid","name","asciiname","alternatenames","lat","lon",
                    "feature class","feature code","country code","cc2",
                    "admin1 code","admin2 code","admin3 code","admin4 code","population",
                    "elevation","dem","timezone","modification date"]
geo = pd.read_csv("AR.txt",sep='\t',index_col=0,names=geo_column_names,)

# Muestra de datos relevantes
print("Shape de la base de datos: {}".format(geo.shape))
print("Muestra de datos de la DB de geonames")
display(geo[["name","lat","lon"]].sample(5))

# Joineamos las tablas por el id de geoname
geodf=df.join(geo[["lat","lon"]],on="geonames_id",rsuffix="_geo")

# Imprimo resultados encontrados
print("")
print("Encontramos datos de latitud y longitud que no teniamos para {} observaciones".format(geodf.loc[(geodf["lat"].isnull()) & (~geodf["lat_geo"].isnull())].shape[0]))
print("No pudimos encontrar nuevos datos de latitud y longitud para {} observaciones".format(geodf.loc[(~geodf["lat"].isnull()) & (geodf["lat_geo"].isnull())].shape[0]))
print("Nuestro Dataset original podria quedar solo con {} observaciones sin latitud y longitud".format(geodf.loc[(geodf["lat"].isnull()) & (geodf["lat_geo"].isnull())].shape[0]))
print("")
print("Muestra de resultados con datos existentes y encontrados en Base de geonames")
geodf.loc[(~geodf["lat"].isnull()) & (~geodf["lat_geo"].isnull()),["place_with_parent_names","geonames_id","lat-lon","lat","lon","lat_geo","lon_geo"]].sample(10)

#CONCLUSIÓN 

Shape de la base de datos: (49650, 18)
Muestra de datos de la DB de geonames


,name,lat,lon
geonameid,,,
3854116,Cerro Huaiquerías,-35.55000,-68.28333
3846670,Loma Rica,-28.43333,-64.70000
3852116,Estancia La Chacra,-34.81512,-62.53215
11364062,Hito XI-Treinta Paso de Las Leñas,-34.46354,-70.07477
3840539,Pozo de la Gran Bestia,-27.36667,-60.70000



Encontramos datos de latitud y longitud que no teniamos para 43204 observaciones
No pudimos encontrar nuevos datos de latitud y longitud para 10825 observaciones
Nuestro Dataset original podria quedar solo con 8346 observaciones sin latitud y longitud

Muestra de resultados con datos existentes y encontrados en Base de geonames


,place_with_parent_names,geonames_id,lat-lon,lat,lon,lat_geo,lon_geo
22188,|Argentina|Buenos Aires Costa Atlántica|Mar de...,3430863.0,"-38.0135013,-57.5371288",-38.013501,-57.537129,-38.00042,-57.55620
55201,|Argentina|Capital Federal|Villa Pueyrredón|,3429703.0,"-34.5881753,-58.5013924",-34.588175,-58.501392,-34.58333,-58.50000
36974,|Argentina|Capital Federal|Palermo|,3430234.0,"-34.5898948,-58.4183366",-34.589895,-58.418337,-34.58856,-58.43053
162,|Argentina|Buenos Aires Costa Atlántica|Mar de...,3430863.0,"-38.0131731,-57.5509598",-38.013173,-57.550960,-38.00042,-57.55620
44940,|Argentina|Córdoba|Córdoba|,3860259.0,"-31.3748036524,-64.2929839343",-31.374804,-64.292984,-31.41350,-64.18105
38670,|Argentina|Bs.As. G.B.A. Zona Oeste|Tres de Fe...,3428075.0,"-34.5652803,-58.5157358",-34.565280,-58.515736,-34.60379,-58.54610
41703,|Argentina|Buenos Aires Costa Atlántica|Mar de...,3430863.0,"-37.984512,-57.5449325",-37.984512,-57.544933,-38.00042,-57.55620
57032,|Argentina|Córdoba|Córdoba|,3860259.0,"-31.451814,-64.137956",-31.451814,-64.137956,-31.41350,-64.18105
27113,|Argentina|Capital Federal|Congreso|,3435259.0,"-34.6093483668,-58.3850130612",-34.609348,-58.385013,-34.60948,-58.39225
7082,|Argentina|Buenos Aires Costa Atlántica|Mar de...,3430863.0,"-37.9920633,-57.5479535",-37.992063,-57.547954,-38.00042,-57.55620


### Trabajo exploratorio con url de imágenes:
##### Trabajo con duplicación de imágenes

In [ ]:
import requests
import hashlib

# Analizamos los primeros 100 casos, leemos la imagen, y le hacemos un hash para comparar a ver si es exacta igual a otra.
r = pd.DataFrame([
    hashlib.md5(requests.get(url = df.loc[x,"image_thumbnail"], params = []).text.encode()).hexdigest() for x in range(100)
])
# Imprimo  cantidad de duplicados
print("Cantidad de imagenes duplicadas (con distinta url) de las primeras 100 observaciones {} ".format(r.duplicated().sum()))

#Concluimos que no es una variable significativa ya que cuenta con imágenes duplicadas

##  2. Normalizar, corregir y rellenar información

En este bloque pretendemos llevar a cabo lo concluido a partir del análisis exploratorio. El objetivo es estandarizar la información del dataset, corrigiendo y completando datos faltantes. 

### Trabajos realizados sobre superficie de las propiedades:

##### Corregimos m2 que encontramos invertidos entre superficie total y superficie cubierta:

In [22]:
#Creo columna temporal_dos para filtrar subconjunto de datos relevantes a invertir
df['temporal_dos'] = (df.surface_total_in_m2 < df.surface_covered_in_m2)
print("Cantidad de registros a invertir entre Superficies total y cubierta: {}".format(df['temporal_dos'].sum()))

#Creo columna temporal para guardar datos
df['temporal'] = df.surface_total_in_m2 

#Paso valores de superficie cubierta a superficie total
df.loc[df['temporal_dos'],'surface_total_in_m2'] = df.loc[df['temporal_dos'],'surface_covered_in_m2']

#Paso valores de superficie total a superficie cubierta
df.loc[df['temporal_dos'], 'surface_covered_in_m2'] = df.loc[df['temporal_dos'], 'temporal']

#Recreamos la columna temporal para ver si siguen existiendo valores invertidos
df['temporal_dos'] = (df.surface_total_in_m2 < df.surface_covered_in_m2)
print("Cantidad de registros que siguen invertidos: {}".format(df['temporal_dos'].sum()))

#Dropeamos las temporales
df.drop('temporal', axis=1)
df.drop('temporal_dos', axis=1);

Cantidad de registros a invertir entre Superficies total y cubierta: 1106
Cantidad de registros que siguen invertidos: 0


##### Limpiamos nulls en variables de superficie:

In [23]:
# Los metros en cero en superficie totales los ponemos en null
print("Valores M2 cubierto en cero puestos en Nan: {}".format((df["surface_covered_in_m2"] == 0).sum()))
print("Valores M2 totales en cero puestos en Nan: {}".format((df["surface_total_in_m2"] == 0).sum()))
df.loc[(df["surface_total_in_m2"] == 0),["surface_total_in_m2"]] = np.nan
df.loc[(df["surface_covered_in_m2"] == 0),["surface_covered_in_m2"]] = np.nan
print("-------------")

#Revisamos valores antes del reemplazo
print("Antes del reemplazo")
print("Nulos en totales: {}".format(df['surface_total_in_m2'].isnull().sum()))
print("Nulos en cubiertos: {}".format(df['surface_covered_in_m2'].isnull().sum()))
print("Nulos en ambos al mismo tiempo: {}".format(df.loc[(df['surface_covered_in_m2'].isnull()) & (df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Nulos totales y no en cubiertos: {}".format(df.loc[(~df['surface_covered_in_m2'].isnull()) & (df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Nulos cubierto y no en totales: {}".format(df.loc[(df['surface_covered_in_m2'].isnull()) & (~df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Valores iguales: {}".format(df.loc[df['surface_total_in_m2'] == df['surface_covered_in_m2'],"surface_covered_in_m2"].count()))


# relleno los m2totales faltantes con los cubiertos
df.loc[(~df['surface_covered_in_m2'].isnull()) & ( df['surface_total_in_m2'].isnull()) ,"surface_total_in_m2"] = df["surface_covered_in_m2"]
# relleno los m2cubiertos faltantes con los totales
df.loc[( df['surface_covered_in_m2'].isnull()) & (~df['surface_total_in_m2'].isnull()) ,"surface_covered_in_m2"] = df["surface_total_in_m2"]
print("-------------")

#Revisamos valores despues del reemplazo
print("Despues del reemplazo")
print("Nulos en totales: {}".format(df['surface_total_in_m2'].isnull().sum()))
print("Nulos en cubiertos: {}".format(df['surface_covered_in_m2'].isnull().sum()))
print("Nulos en ambos al mismo tiempo: {}".format(df.loc[(df['surface_covered_in_m2'].isnull()) & (df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Nulos totales y no en cubiertos: {}".format(df.loc[(~df['surface_covered_in_m2'].isnull()) & (df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Nulos cubierto y no en totales: {}".format(df.loc[(df['surface_covered_in_m2'].isnull()) & (~df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Valores iguales: {}".format(df.loc[df['surface_total_in_m2'] == df['surface_covered_in_m2'],"surface_covered_in_m2"].count()))


Valores M2 cubierto en cero puestos en Nan: 2
Valores M2 totales en cero puestos en Nan: 383
-------------
Antes del reemplazo
Nulos en totales: 39711
Nulos en cubiertos: 19909
Nulos en ambos al mismo tiempo: 12752
Nulos totales y no en cubiertos: 26959
Nulos cubierto y no en totales: 7157
Valores iguales: 24173
-------------
Despues del reemplazo
Nulos en totales: 12752
Nulos en cubiertos: 12752
Nulos en ambos al mismo tiempo: 12752
Nulos totales y no en cubiertos: 0
Nulos cubierto y no en totales: 0
Valores iguales: 58289


##### Extraemos cantidad de ambientes faltantes de los títulos:

In [24]:
#Creamos una regex  y la corremos en titulo y en descripcion para ver que encuentra
pat_ambientes = r'\b(\d{1,2})\s*amb'

#Extraemos los datos del título a la nueva columna amb_tit
df['amb_tit'] = df.title.str.extract(pat_ambientes, re.IGNORECASE, expand=True).astype(np.float)

#Verificamos los valores extraídos
df.loc[~(df.amb_tit.isnull())].filter(['amb_tit']).sort_values('amb_tit').amb_tit.unique()

#Revisamos cuantos valores extrajimos
df.loc[~(df.amb_tit.isnull())].title.count()


21799

###### Llenamos los `NaN` con los valores obtenidos en `amb_tit`

In [25]:
#Revisamos cuantos podemos salvar
df.loc[(df.rooms.isnull()) & (~df.amb_tit.isnull())]

#Llenamos los NaN's verificando cuales no tienen valores en rooms
display(df.loc[~df.rooms.isnull()].shape)

#Hacemos el fillna
df.rooms.fillna(df.amb_tit, inplace=True)

df.loc[~df.rooms.isnull()].shape (QUEDAN CON DATOS X REGISTROS)


(47390, 33)

### Trabajos realizados sobre el precio de las propiedades:

In [ ]:
# Dropeamos los planes ahorro
dfprecio = dfprecio.loc[~dfprecio['title'].str.contains(r'cuotas')];

##### Sumarizamos los resultados de los grupos de captura y creamos un nuevo dataframe con esos datos:


In [ ]:
df_precio_title['sumat'] = df_precio_title[1] + df_precio_title[2] + df_precio_title[3]
df_precio_desc['sumad'] = df_precio_desc[1] + df_precio_desc[2] + df_precio_desc[3]
df_precio_title = df_precio_title.iloc[:,[0,4]]
df_precio_desc = df_precio_desc.iloc[:,[0,4]]
df_precio_title['sumat'].fillna(0, inplace = True);

#### Eliminamos NaNs y strings vacios:

In [ ]:
df_precio_title.replace("", 0);
df_precio_desc['sumad'] = df_precio_desc.sumad.str.strip();
df_precio_desc.replace("", 0, inplace=True)
df_precio_desc['sumad'].fillna(0 ,inplace=True)
dfpredesc2 = pd.merge(df_precio_desc, df_precio_title, how='outer', on=df_precio_desc.index)

#### Unificamos los datos obtenidos via regex:

In [ ]:
dfpredesc2['sumad'] = pd.to_numeric(dfpredesc2['sumad'])
dfpredesc2['sumat'] = pd.to_numeric(dfpredesc2['sumat'])
dfpredesc2['final'] = dfpredesc2[['sumad','sumat']].max(axis=1)
dfpredesc2.loc[(dfpredesc2['0_x'].isnull())&(dfpredesc2['0_y'].notnull()),'0_x']=dfpredesc2['0_y'];
df_precio = dfpredesc2.loc[(dfpredesc2['final']!= 0) & (dfpredesc2['final']!= 1)].filter(['0_x','0_y', 'final'])
del df_precio['0_y']
df_precio.rename(columns = {'0_x':'moneda'}, inplace=True)
dfprecio['price'].fillna(df_precio['final'], inplace=True)
dfprecio['currency'].fillna(df_precio['moneda'], inplace=True)


#### Incorporamos los datos obtenidos al Data Frame original:

In [ ]:
dfprecio.rename(columns={'price':'precio_regex','currency':'moneda'}, inplace = True)
df = df.join(dfprecio[['precio_regex', 'moneda']])


### Trabajos realizados sobre la ubicación de las propiedades: 

##### Desagregamos los datos contenidos en place_with_parent_names: 

In [26]:
#Spliteamos columna place_with_parent_names y nombramos a las nuevas columnas
place_split = df.place_with_parent_names.str.split('|', expand=True).rename({1:'pais', 2:'provincia',
                                                               3:'localidad', 4:'barrio'}, axis=1).drop([0,5,6], axis=1)
place_split.loc[(place_split.barrio == ''), 'barrio'] = np.nan

#Lo adjuntamos al df original
df = df.join(place_split)
df

#Eliminamos las columnas ahora innecesarias 
df.drop(["place_with_parent_names", "Unnamed: 0"], axis=1, inplace=True)


## 3. Quitar todo lo que no nos sirve

En esta etapa de trabajo eliminamos todos aquellos datos que no serán necesarios para la construicción de nuestro modelo de regresión.

##### Eliminamos duplicados: 

In [ ]:
# Muestro forma inicial
display(df.shape)

# Buscar índices de registros duplicados (sin tener en cuenta las urls y la 1er columna de autonumerico)
duplicados=df.loc[df.drop("Unnamed: 0",axis=1).drop("properati_url",axis=1).drop("image_thumbnail",axis=1).duplicated(keep="last")]
print("Registros duplicados: {}".format(duplicados["operation"].count()))


# DROP duplicados
df.drop(duplicados.index, inplace=True)
display(df.shape)

##### Eliminamos columnas redundantes que no agregan al modelo de predicción:

In [ ]:
# Eliminamos columnas sin uso
def drop_column(column,df):
    try:
        df.drop(column,axis=1,inplace=True)
        print("Dropeando columna {} ".format(column));
    except:
        print("Columna {} ya dropeada ".format(column)) ;
    
# Muestro forma inicial
display(df.shape)

# properati_url: no tiene ningun uso de valor predictivo
drop_column("properati_url",df)

# image_thumbnail: como se vió antes, hay imágenes duplicadas para departamentos distintos
drop_column("image_thumbnail",df)

# unnamed 0: replica el indice en cada linea
drop_column("Unnamed: 0",df)

# operation: siempre es venta, no suma nada al modelo
drop_column("operation",df)

# country_name: siempres es argentina, no suma a modelo
drop_column("country_name",df)


display(df.shape)

##### Eliminamos observaciones contienen la frase "en pozo" en la descrpción

In [ ]:
#Observamos cuantas observaciones son
df['pozo'] = df.description.str.extract(r'(\ben\spozo)', re.IGNORECASE, expand=True)
df.loc[~(df.pozo.isnull())]

#Eliminamos dichos datos 
df.drop(df.loc[~(df.pozo.isnull())].index, axis=0, inplace=True)

## 4. Calcular las variables dummies y mostrar los resultados